# 应用外点罚函数方法和拉格朗日乘子法求解约束最优化问题

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import math

def targetfunction(x):# x是列向量
    return (x[0]-1)**2+(x[0]-x[1])**2+(x[1]-x[2])**4

def limitation(x):# 约束条件，x是列向量
    if -10 <= x[0] <= 10 and -10 <= x[1] <= 10 and -10 <= x[2] <= 10:
        if x[0]*(1+x[1]**2)+x[2]**4-4-3*math.sqrt(2)==0:
            return True
    return False

def equalization(x):# 等式约束条件，x是列向量
    return (x[0]*(1+x[1]**2)+x[2]**4-4-3*math.sqrt(2.0))**2

def inequalization(x):# 不等式约束条件，x是列向量
    return (min(x[0]+10,0))**2+(min(10-x[0],0))**2+(min(x[1]+10,0))**2+(min(10-x[1],0))**2+(min(x[2]+10,0))**2+(min(10-x[2],0))**2

def penaltyfunction(x,miu):# 外点罚函数
    return targetfunction(x)+0.5*miu*(equalization(x)+inequalization(x))

def grad(f, x):
    '''
    求梯度
    :param f: 函数
    :param x: 向量
    :return: 梯度向量
    '''
    delta=0.0000001
    gradmatrix=np.zeros(x.shape)
    fx=f(x)
    it=np.nditer(x,flags=['multi_index'],op_flags=['readwrite'])
    while not it.finished:
        ix=it.multi_index
        old_value=(np.float64)(x[ix])
        x[ix]=(np.float64)(old_value+delta)
        fxd=f(x)
        gradmatrix[ix]=(fxd-fx)/delta
        x[ix]=old_value
        it.iternext()
    return np.mat(gradmatrix)

def Hessian(f,x):
    '''
    求Hessian矩阵
    :param f: 函数
    :param x: 初始点
    :param epsilon: epsilon
    :return: f在x处的Hessian矩阵
    '''
    delta=1e-3
    n=np.size(x)
    HessianMatrix=np.zeros((n,n))
    gx0=grad(f,x)
    for i in range(0,n):
        old_value = (np.float64)(x[i,0])
        x[i,0] = (np.float64)(old_value + delta)
        gxk=grad(f,x)
        for j in range(0,n):
            HessianMatrix[i,j]=(np.float64)((gxk[j,0]-gx0[j,0])/delta)
        x[i,0]=old_value
    return np.mat(HessianMatrix)

def Armijo(f,xk,dk,rho):# 非精确线搜索，采用armijo方法
    alpha=1
    beta=rho
    sigma=0.2
    while True:
        if f(xk+alpha*dk)<=f(xk)+sigma*alpha*grad(f,xk).T*dk:
            break
        alpha*=beta
    return alpha

def dampnewton(function,x,epsilon):# 阻尼牛顿法
    xk=x
    gk=grad(function,xk)
    while(np.linalg.norm(gk)>epsilon):
        if (np.linalg.det(Hessian(function,xk))<epsilon):
            dk=-gk
        else:
            dk=-np.linalg.inv(Hessian(function,xk))*gk
        alpha=Armijo(function,xk,dk,0.5)
        xk=xk+alpha*dk
        gk=grad(function,xk)
    return xk

def Graddecent(function,x,epsilon):# 最速梯度法
    xk=x
    gk=grad(function,xk)
    max_iter=10000
    cnt=0
    while(np.linalg.norm(gk)>epsilon):
        dk=-grad(function,xk)
        alpha=Armijo(function,xk,dk,0.5)
        xk=xk+dk*alpha
        gk=grad(function,xk)
        cnt+=1
        if cnt==max_iter:
            break
    return xk

def penaltyfunctionmethod(x0,miu0,epsilon):
    start=time.time()
    cnt=0
    xk=x0
    miuk=miu0
    while True:
        xk=dampnewton(lambda x:penaltyfunction(x,miuk),xk,epsilon)
        if equalization(xk)+inequalization(xk)<epsilon:
            break
        miuk=miuk*10
        cnt+=1
        print("cnt=",cnt)
    end=time.time()-start
    return xk,end,cnt

X=np.mat([2.0,2.0,2.0]).T
Xtrue=np.mat([1.104859024,1.196674194,1.535262257]).T
miu=1
epsilon=1e-6
xk,end,cnt=penaltyfunctionmethod(X,miu,epsilon)
print("最优解为：",xk.T)
print("最优值为：",targetfunction(xk))
print("迭代次数为：",cnt)
print("运行时间为：",end)
print("理想最优解：",targetfunction(Xtrue))

cnt= 1
cnt= 2
最优解为： [[1.10492417 1.19674089 1.53523167]]
最优值为： [[0.03256705]]
迭代次数为： 2
运行时间为： 0.1712644100189209
理想最优解： [[0.0325682]]


可知，外点罚函数法运行时间较长，可能与采用armijo非精确线搜索方法相关，同时也由于此方法有数值困难。导致使用最速梯度法运行时间能够达到370s，过于低效率，使用阻尼牛顿法有明显改善。

# 拉格朗日乘子法

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import math

def targetfunction(x):# x是列向量
    return (x[0]-1)**2+(x[0]-x[1])**2+(x[1]-x[2])**4

def limitation(x):# 约束条件，x是列向量
    if -10 <= x[0] <= 10 and -10 <= x[1] <= 10 and -10 <= x[2] <= 10:
        if x[0]*(1+x[1]**2)+x[2]**3-4-3*math.sqrt(2)==0:
            return True
    return False

def equalization(x):# 等式约束条件，x是列向量
    return x[0]*(1+x[1]**2)+x[2]**4-4-3*math.sqrt(2)

def inequalization(x):# 不等式约束条件，x是列向量
    return (min(x[0]+10,0))**2+(min(10-x[0],0))**2+(min(x[1]+10,0))**2+(min(10-x[1],0))**2+(min(x[2]+10,0))**2+(min(10-x[2],0))**2

def ita(lamda,miu):
    return lamda/miu

def fi(ci,x,lamda,miu):
    if ci(x)-ita(lamda,miu)>=0:
        return -0.5*miu*ita(lamda,miu)**2
    else:
        return 0.5*miu*((ci(x)-ita(lamda,miu))**2-ita(lamda,miu)**2)
    
def interfi(x,lamda,miu):# 所有不等式约束条件转化成的fi
    limit1=lambda x:10-x
    limit2=lambda x:x+10
    return fi(limit1,x[0],lamda[0],miu)+fi(limit2,x[0],lamda[1],miu)+fi(limit1,x[1],lamda[2],miu)+fi(limit2,x[1],lamda[3],miu)+fi(limit1,x[2],lamda[4],miu)+fi(limit2,x[2],lamda[5],miu)

def lagrangefunction(x,lamda,miu):# 拉格朗日函数
    return targetfunction(x)-lamda[0]*equalization(x)+0.5*miu*(equalization(x)**2)+interfi(x,lamda[1:],miu)

def endtest(x,lamda,miu,epsilon):# 终止条件
    limit1=lambda x:10-x
    limit2=lambda x:x+10
    return math.sqrt(targetfunction(x)**2+min(ita(lamda[1],miu),limit1(x[0]))**2+min(ita(lamda[2],miu),limit2(x[0]))**2+min(ita(lamda[3],miu),limit1(x[1]))**2+min(ita(lamda[4],miu),limit2(x[1]))**2+min(ita(lamda[5],miu),limit1(x[2]))**2+min(ita(lamda[6],miu),limit2(x[2]))**2)<=epsilon

def endtest2(x,epsilon):# 改进后的终止条件
    limit1=lambda x:10-x
    limit2=lambda x:x+10
    return np.linalg.norm(equalization(x))+np.linalg.norm(np.mat([min(limit1(x[0]),0),min(limit2(x[0]),0),min(limit1(x[1]),0),min(limit2(x[1]),0),min(limit1(x[2]),0),min(limit2(x[2]),0)]).T) <=epsilon

def lamdaiter(x,lamda,miu):
    limit1=lambda x:10-x
    limit2=lambda x:x+10
    lamda[0]=lamda[0]-miu*equalization(x)
    lamda[1]=-miu*min(limit1(x[0])-ita(lamda[1],miu),0)
    lamda[2]=-miu*min(limit2(x[0])-ita(lamda[2],miu),0)
    lamda[3]=-miu*min(limit1(x[1])-ita(lamda[3],miu),0)
    lamda[4]=-miu*min(limit2(x[1])-ita(lamda[4],miu),0)
    lamda[5]=-miu*min(limit1(x[2])-ita(lamda[5],miu),0)
    lamda[6]=-miu*min(limit2(x[2])-ita(lamda[6],miu),0)
    return lamda

def grad(f, x):
    '''
    求梯度
    :param f: 函数
    :param x: 向量
    :return: 梯度向量
    '''
    delta=0.0000001
    gradmatrix=np.zeros(x.shape)
    fx=f(x)
    it=np.nditer(x,flags=['multi_index'],op_flags=['readwrite'])
    while not it.finished:
        ix=it.multi_index
        old_value=(np.float64)(x[ix])
        x[ix]=(np.float64)(old_value+delta)
        fxd=f(x)
        gradmatrix[ix]=(fxd-fx)/delta
        x[ix]=old_value
        it.iternext()
    return np.mat(gradmatrix)

def Hessian(f,x):
    '''
    求Hessian矩阵
    :param f: 函数
    :param x: 初始点
    :param epsilon: epsilon
    :return: f在x处的Hessian矩阵
    '''
    delta=1e-3
    n=np.size(x)
    HessianMatrix=np.zeros((n,n))
    gx0=grad(f,x)
    for i in range(0,n):
        old_value = (np.float64)(x[i,0])
        x[i,0] = (np.float64)(old_value + delta)
        gxk=grad(f,x)
        for j in range(0,n):
            HessianMatrix[i,j]=(np.float64)((gxk[j,0]-gx0[j,0])/delta)
        x[i,0]=old_value
    return np.mat(HessianMatrix)

def Armijo(f,xk,dk,rho):# 非精确线搜索，采用armijo方法
    alpha=1
    beta=rho
    sigma=0.5
    max_iter=5000
    cnt=0
    while True:
        if f(xk+alpha*dk)<=f(xk)+sigma*alpha*grad(f,xk).T*dk:
            break
        alpha*=beta
        cnt+=1
        if cnt==max_iter:
            break
    return alpha

def dampnewton(function,x,epsilon):
    xk=x
    gk=grad(function,xk)
    max_iter=10000
    cnt=0
    while(np.linalg.norm(gk)>epsilon):
        if (np.linalg.det(Hessian(function,xk))<epsilon):
            dk=-gk
        else:
            dk=-np.linalg.inv(Hessian(function,xk))*gk
        alpha=Armijo(function,xk,dk,0.5)
        xk=xk+alpha*dk
        gk=grad(function,xk)
        cnt+=1
        if abs(function(xk)-function(xk-alpha*dk))<=epsilon:
            break
        if cnt==max_iter:
            break
    return xk

def Graddecent(function,x,epsilon):
    xk=x
    gk=grad(function,xk)
    max_iter=10000
    cnt=0
    while(np.linalg.norm(gk)>epsilon):
        dk=-grad(function,xk)
        alpha=Armijo(function,xk,dk,0.5)
        xk=xk+dk*alpha
        gk=grad(function,xk)
        cnt+=1
        if cnt==max_iter:
            break
    return xk

def lagrangemethod(x0,lamda0,miu0,epsilon,rou):
    start=time.time()
    max_iter=100
    cnt=0
    xk=x0
    lamdak=lamda0
    miuk=miu0
    while True:
        xlast=xk
        xk=dampnewton(lambda x:lagrangefunction(x,lamdak,miuk),xk,epsilon)
        lamdak=lamdaiter(xk,lamdak,miuk)
        cnt+=1
        print("cnt=",cnt)
        # if endtest(xk,lamdak,miuk,epsilon):
        if endtest2(xk,epsilon):
            break
        miuk=miuk*rou
        if cnt==max_iter:
            break
    end=time.time()-start
    return xk,end,cnt    


X=np.mat([2.0,2.0,2.0]).T
Xtrue=np.mat([1.104859024,1.196674194,1.535262257]).T
lamda=np.mat([1.0,1.0,1.0,1.0,1.0,1.0,1.0]).T
miu=1
epsilon=1e-6
rou=2
xk,end,cnt=lagrangemethod(X,lamda,miu,epsilon,rou)
print("最优解为：",xk.T)
print("最优值为：",targetfunction(xk))
print("迭代次数为：",cnt)
print("运行时间为：",end)
print("理想最优解：",targetfunction(Xtrue))


cnt= 1
cnt= 2
cnt= 3
cnt= 4
cnt= 5
cnt= 6
cnt= 7
cnt= 8
cnt= 9
cnt= 10
cnt= 11
cnt= 12
cnt= 13
cnt= 14
最优解为： [[1.10488916 1.19670324 1.53525192]]
最优值为： [[0.03256821]]
迭代次数为： 14
运行时间为： 0.63747239112854
理想最优解： [[0.0325682]]


在低精度1e-4情况下，拉格朗日乘子法有较多的迭代次数，但是运行时间得到了提升。

在高精度1e-6情况下，拉格朗日乘子法在运行时间和迭代次数上均大于外点罚函数法，推测由于终止条件没有较好改进所致。

两种方法都能得到与最优解极其接近的结果，效果较好。